In [5]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

     |████████████████████████████████| 4.3MB 4.7MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 4.6MB/s 


In [0]:
import pandas as pd
import numpy as np

In [0]:
from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae 
from sklearn.model_selection import cross_val_score, KFold

import eli5 
from eli5.sklearn import PermutationImportance


In [8]:
cd "drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car"


[Errno 2] No such file or directory: 'drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car'
/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car


In [9]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat: #jeśli cecha zawiera końcówkę _cat, nadpisz wartość
    df[feat] = factorized_values
  else:
      df[feat + SUFFIX_CAT] = factorized_values

In [15]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]#nie chcemy tego gdy wartościa jest liczba, czyli w kolumnach z ceną
len(cat_feats)

151

##Decision Tree

In [0]:
def run_model(model, feats):
 X = df[cat_feats].values
 y = df['price_value'].values

 #model = DecisionTreeRegressor(max_depth=5)
# scores = cross_val_score(model, X, y, cv=3, scoring = 'neg_mean_absolute_error')
 return np.mean(scores), np.std(scores)

In [20]:
run_model(DecisionTreeRegressor(max_depth=5), cat_feats)

(-19747.81093847179, 75.95212671816309)

##Random Forest

In [21]:
model = RandomForestRegressor(max_depth=5, n_estimators=50, random_state=0)
run_model(model, cat_feats)

(-19747.81093847179, 75.95212671816309)

##XGBoost

In [41]:
xgb_params = {
    'max_depth': 5,
    'n_estimators': 50,
    'learning_rate': 0.1,
    'seed' :0
}

run_model(xgb.XGBRegressor(**xgb_params), cat_feats)

(-19747.81093847179, 75.95212671816309)

In [25]:
m = xgb.XGBRegressor(max_depth=5, n_estimators=50,learning_rate=0.1,seed=0)
m.fit(X, y)

#ktore cechy sa istotne wg xgboosta
inp= PermutationImportance(m).fit(X, y)
eli5.show_weights(inp, feature_names = cat_feats)

[23:26:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1274 ± 0.0025,param_napęd__cat
0.1160 ± 0.0023,param_stan__cat
0.1127 ± 0.0020,param_rok-produkcji__cat
0.0563 ± 0.0009,param_faktura-vat__cat
0.0484 ± 0.0016,param_moc__cat
0.0407 ± 0.0009,param_skrzynia-biegów__cat
0.0252 ± 0.0006,param_marka-pojazdu__cat
0.0244 ± 0.0007,feature_kamera-cofania__cat
0.0233 ± 0.0004,param_typ__cat
0.0194 ± 0.0011,param_pojemność-skokowa__cat


In [27]:
len(cat_feats)

151

In [0]:
feats=['param_napęd__cat','param_stan__cat','param_rok-produkcji__cat','param_faktura-vat__cat','param_moc__cat','param_skrzynia-biegów__cat','param_marka-pojazdu__cat','feature_kamera-cofania__cat','param_typ__cat','param_pojemność-skokowa__cat','seller_name__cat','param_wersja__cat','param_model-pojazdu__cat','feature_wspomaganie-kierownicy__cat','feature_system-start-stop__cat','param_kod-silnika__cat','feature_asystent-pasa-ruchu__cat','feature_łopatki-zmiany-biegów__cat','feature_czujniki-parkowania-przednie__cat','feature_światła-led__cat']

In [33]:
#sprawdzamy czy jak zmienimy ilość cech z 151 na 20 (te uznane za ważne) to wynik się zmeni
len(feats)

20

In [34]:
run_model(xgb.XGBRegressor(**xgb_params), feats)

(-19747.810938471786, 75.95212671816479)

In [37]:
#analizujemy co jest w kolumnach. Poszukujemy zmienne numeryczne
df['param_rok-produkcji'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76])

In [0]:
df['param_rok-produkcji__cat'].unique()
#po zastosowani funki cat te lata się dubią, tracimy informację , możemy je zopymalizować troche

In [42]:
df['param_rok-produkcji']=df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))
feats=['param_napęd__cat','param_stan__cat','param_rok-produkcji','param_faktura-vat__cat','param_moc__cat','param_skrzynia-biegów__cat','param_marka-pojazdu__cat','feature_kamera-cofania__cat','param_typ__cat','param_pojemność-skokowa__cat','seller_name__cat','param_wersja__cat','param_model-pojazdu__cat','feature_wspomaganie-kierownicy__cat','feature_system-start-stop__cat','param_kod-silnika__cat','feature_asystent-pasa-ruchu__cat','feature_łopatki-zmiany-biegów__cat','feature_czujniki-parkowania-przednie__cat','feature_światła-led__cat']
run_model(xgb.XGBRegressor(**xgb_params), feats)

(-19747.81093847179, 75.95212671816309)

In [46]:
df['param_moc']=df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]) )
feats=['param_napęd__cat','param_stan__cat','param_rok-produkcji','param_faktura-vat__cat','param_moc','param_skrzynia-biegów__cat','param_marka-pojazdu__cat','feature_kamera-cofania__cat','param_typ__cat','param_pojemność-skokowa__cat','seller_name__cat','param_wersja__cat','param_model-pojazdu__cat','feature_wspomaganie-kierownicy__cat','feature_system-start-stop__cat','param_kod-silnika__cat','feature_asystent-pasa-ruchu__cat','feature_łopatki-zmiany-biegów__cat','feature_czujniki-parkowania-przednie__cat','feature_światła-led__cat']
run_model(xgb.XGBRegressor(**xgb_params), feats) #poniewaz nadpisujmy param_moc w feats zmieniamy z wersji __cat na param_moc

(-19747.81093847179, 75.95212671816309)

In [48]:
df['param_pojemność-skokowa'].unique()

array(['898 cm3', '1 560 cm3', '3 000 cm3', ..., '5 992 cm3', '1 966 cm3',
       '142 280 cm3'], dtype=object)

In [51]:
df['param_pojemność-skokowa']=df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split('cm')[0].replace(' ','')) )
feats=['param_napęd__cat','param_stan__cat','param_rok-produkcji','param_faktura-vat__cat','param_moc','param_skrzynia-biegów__cat','param_marka-pojazdu__cat','feature_kamera-cofania__cat','param_typ__cat','param_pojemność-skokowa','seller_name__cat','param_wersja__cat','param_model-pojazdu__cat','feature_wspomaganie-kierownicy__cat','feature_system-start-stop__cat','param_kod-silnika__cat','feature_asystent-pasa-ruchu__cat','feature_łopatki-zmiany-biegów__cat','feature_czujniki-parkowania-przednie__cat','feature_światła-led__cat']
run_model(xgb.XGBRegressor(**xgb_params), feats)

(-19747.810938471786, 75.95212671816479)